In [1]:
import quinn
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


22/07/07 12:32:08 WARN Utils: Your hostname, fossa-dsa-001 resolves to a loopback address: 127.0.1.1; using 192.168.0.111 instead (on interface wlp3s0)
22/07/07 12:32:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/07 12:32:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from pyspark.sql import SparkSession
from datetime import date
import quinn
from pyspark.sql.functions import lit


def dashes_to_underscores(s):
    return s.replace("-", "_")


#create spark session
spark = SparkSession.builder.getOrCreate()
#create df
daily_df = spark.read.csv(
    'Daily_Country_Wise_Confirmed_Cases.csv', header=True)
#perform transformations
dataset_df = daily_df.transform(
    quinn.with_columns_renamed(dashes_to_underscores))
#create pandas df and set index
pandas_df = daily_df.toPandas()
pandas_df.set_index('Country', inplace=True)
#transpose and reset index
transpose_df = pandas_df.transpose()
transpose_df.reset_index(inplace=True)
#create spark df
spark_df = spark.createDataFrame(transpose_df)
spark_df = spark_df.withColumnRenamed('index', 'Date')

spark_df.write.csv(f"Daily_CC_{date.today()}")


In [20]:
import pandas as pd
import numpy as np

# turn the spark df into pandas df
pd_pox = dataset_df.toPandas()

pd_pox = pd_pox.fillna(0) 
pd.isnull(pd_pox).sum()
# pd_pox.
pd_pox.to_csv('../data/daily_cases.csv')
# drop the silly enumerated index and replace it with count=rty names
pd_pox.set_index('Country', drop=True, inplace=True)
# this will take our row and column indices and flip 'em
# flipped_df = pd_pox.transpose()


# # set df data types to floats and then into ints
# flipped_df = flipped_df.astype(float).astype('Int64')
# # see if we have any null values
# pd.isnull(flipped_df).sum()
# # replace them with 0
# flipped_df = flipped_df.fillna(0)

# pd.isnull(flipped_df).sum()

pd_pox.index


Index(['England', 'Portugal', 'Spain', 'United States', 'Canada', 'Sweden',
       'Italy', 'France', 'Belgium', 'Australia', 'Germany', 'Netherlands',
       'Israel', 'Switzerland', 'Greece', 'Austria', 'Argentina', 'Denmark',
       'Morocco', 'Slovenia', 'Scotland', 'Czech Republic',
       'United Arab Emirates', 'Finland', 'Wales', 'Northern Ireland', 'Sudan',
       'Bolivia', 'Iran', 'Ecuador', 'Malta', 'Ireland', 'Mexico', 'Pakistan',
       'French Guiana', 'Thailand', 'Peru', 'Brazil', 'Malaysia', 'Hungary',
       'Norway', 'Paraguay', 'Costa Rica', 'Gibraltar', 'Mauritius', 'Haiti',
       'Uruguay', 'Latvia', 'Cayman Islands', 'Kosovo', 'Turkey', 'Bahamas',
       'Ghana', 'India', 'Iceland', 'Poland', 'Bangladesh', 'Uganda',
       'Cambodia', 'Malawi', 'Venezuela', 'Romania', 'Georgia', 'Slovakia',
       'Luxembourg', 'Nepal', 'Chile', 'Serbia', 'Lebanon', 'South Korea',
       'Singapore', 'South Africa', 'Taiwan', 'Colombia', 'Croatia',
       'Bulgaria', 'Somalia', 

In [4]:
import pandas as pd

# turn the spark df into pandas df
pd_pox = dataset_df.toPandas()
# type(pd_pox)

# pd_pox.set_index('Country', drop = True, inplace = True)
country_cases = pd_pox.pivot_table(columns = 'Country', values = 'Confirmed_Cases', fill_value = 0)
# pd_pox.info()
# pd_pox.describe
country_cases

# pd.isnull(country_cases)
# pd_pox.index




Country,Argentina,Australia,Austria,Bahamas,Bangladesh,Belgium,Benin,Bolivia,Brazil,Bulgaria,...,Taiwan,Thailand,Turkey,Uganda,United Arab Emirates,United States,Uruguay,Venezuela,Wales,Zambia
Confirmed_Cases,6,12,37,0,0,168,3,0,79,3,...,1,0,1,0,13,559,0,1,15,0


In [1]:
# Imports the Google Cloud client library
import os
from google.cloud import storage

# Instantiates a client

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/fossa/data/dont_worry_about_it/dearHenry.json"
client = storage.Client()


# Retrieve an existing bucket
# https://console.cloud.google.com/storage/browser/hole-in-the/
bucket = client.get_bucket('hole-in-the')
# upload the csv to the bucket
blob = bucket.blob('daily_cases.csv')
blob.upload_from_filename('../data/daily_cases.csv')


In [ ]:
from google.cloud import bigquery
import pandas as pd 
import pandas_gbq

# daily_cases = pd.read_csv('../data/Daily_Country_Wise_Confirmed_Cases.csv')
# cases_worldwide = pd.read_csv('../data/Monkey_Pox_Cases_Worldwide.csv')
daily_cases = pd.read_csv('../data/daily_cases.csv')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/fossa/data/dont_worry_about_it/dearHenry.json"
client = bigquery.Client()
dataset_id = f"{client.project}.daily_cases"
dataset = bigquery.Dataset(dataset_id)
dataset.location = 'us'
dataset = client.create_dataset(dataset, exists_ok = True, timeout =100)
project_id = 'dearliza'
table_id = 'daily_cases.daily_cases_data'

pandas_gbq.to_gbq(daily_cases, table_id, project_id = project_id, if_exists = 'replace', api_method = 'load_csv')